
# Practica 2. Redes Neuronales Convolucionales (CNN)

Hernandez Martinez Mally Samira | Código: 220286113 | Ingenieria de computacion (INCO) 

Seminario de Solucion de Problemas de Inteligencia Artificial II | Seccion D05 I7041


<u>Caso de estudio</u>:

Implementación de un clasificador automático de imágenes deportivas que pueda distinguir entre diferentes disciplinas, como fútbol, baloncesto, atletismo, y natación. Este sistema podría ser utilizado por plataformas de transmisión deportiva para etiquetar contenidos automáticamente en tiempo real, con el fin de mejorar la accesibilidad y búsqueda de contenido por parte de los usuarios.

1.	Preprocesamiento de Datos:

    - Cargar un conjunto de datos de imágenes deportivas, con múltiples clases que representan diferentes deportes (ej. fútbol, baloncesto, atletismo, natación, etc.).

    - Asegurarse de que las imágenes estén etiquetadas correctamente para las distintas disciplinas deportivas.

    - Redimensionar las imágenes a un tamaño adecuado para el modelo CNN (ej. 128x128 o 224x224 píxeles).

    - Normalizar los datos de entrada para mejorar la convergencia del modelo, dividiendo los valores de los píxeles por 255 (para que queden entre 0 y 1).


In [2]:
# Importacion de librerías
import os
import sys
import time
import sklearn
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as ft
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.applications import VGG16 
from keras.preprocessing import image
from keras.utils import load_img, img_to_array 
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator 
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization 
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input, decode_predictions 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import UnidentifiedImageError 
from contextlib import contextmanager
from keras.optimizers import Adam 
from tensorflow import keras
from PIL import ImageFile
from PIL import Image

In [ ]:
# Contexto para suprimir la salida
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

# Definir rutas de datos
train_dir = 'C:/Users/mally/Documents/Proyectos/sem_IA2/CNN/dataset'
test_dir = 'C:/Users/mally/Documents/Proyectos/sem_IA2/CNN/single_test'

# Parámetros
img_size = (128, 128)
batch_size = 32

# Generador de datos con aumento para el entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  #
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


# Suprimir la salida de los generadores
with suppress_stdout():
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

# Información sobre las clases
print(f"Clases detectadas:\n{train_generator.class_indices}")
print(f"Imágenes en el conjunto de entrenamiento: {train_generator.samples}")
print(f"Imágenes en el conjunto de validación: {validation_generator.samples}")
print(f"Imágenes en el conjunto de prueba: {test_generator.samples}")


2.	Definición del Modelo:

a) Prueba 1: Modelo CNN desde cero
- Crear una red CNN personalizada utilizando varias capas convolucionales y de pooling.
- Las capas deben incluir filtros de distintas profundidades (ej. 32, 64, 128), con funciones de activación ReLU para las - capas convolucionales.
- Usar Batch Normalization para estabilizar el entrenamiento y mejorar la generalización del modelo.
- Aplicar capas de MaxPooling para reducir la dimensionalidad.
- Aplicar capas Dropout, para evitar el overffitting.
- Añadir capas densas al final, y la última capa debe tener tantas neuronas como clases deportivas, utilizando la función de activación softmax.
- Compilar el modelo utilizando un optimizador como Adam y la función de pérdida categorical_crossentropy.

In [ ]:
# Modelo CNN personalizado
model_cnn = Sequential()

# Capas convolucionales y de pooling
model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(128, (3, 3), activation='relu'))  
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Capas densas con regularización L2 y Dropout
model_cnn.add(Flatten())
model_cnn.add(Dense(256, activation='relu', kernel_regularizer=l2(0.0001)))  
model_cnn.add(Dropout(0.3))
model_cnn.add(Dense(train_generator.num_classes, activation='softmax'))

# Cargar el modelo guardado
model_cnn = load_model('C:/Users/mally/Documents/Proyectos/sem_IA2/CNN/mejor_modelo.h5')

# Compilar el modelo con tasa de aprendizaje reducida
model_cnn.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Checkpoint para guardar el mejor modelo
checkpoint = ModelCheckpoint('mejor_modelo.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Early stopping para evitar sobreentrenamiento
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_cnn = model_cnn.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,  
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    callbacks=[checkpoint, early_stopping]
)

In [ ]:
# Compilar el modelo con una tasa de aprendizaje más baja
model_cnn.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks: early stopping, checkpoint y reducción de LR
checkpoint = ModelCheckpoint('mejor_modelo.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Entrenar el modelo
history_cnn = model_cnn.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    callbacks=[checkpoint, early_stopping, reduce_lr]
)


b) Prueba 2: Modelo con Transfer Learning
- Utilizar un modelo preentrenado (ej. VGG16, ResNet50, InceptionV3) entrenado en ImageNet, cargando sus pesos preentrenados.   
- Congelar las capas convolucionales del modelo preentrenado para conservar las características ya aprendidas.
- Agregar nuevas capas densas al final del modelo, adaptando la última capa para que tenga tantas neuronas como clases deportivas con la función de activación softmax.
- Compilar este modelo también con un optimizador como Adam y la función de pérdida categorical_crossentropy.

In [ ]:
# Parámetros
img_size = (128,128)  
batch_size = 32

with suppress_stdout():
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

# Cargar el modelo preentrenado VGG16 (sin la capa superior) y congelar sus capas
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Congelar todas las capas convolucionales

# Definir un nuevo modelo secuencial
model_transfer = Sequential()

# Añadir el modelo base (VGG16 preentrenado)
model_transfer.add(base_model)

# Aplanar la salida para conectarla a las capas densas
model_transfer.add(Flatten())

# Añadir una capa completamente conectada (densa)
model_transfer.add(Dense(128, activation='relu'))
model_transfer.add(Dropout(0.5))  # Dropout para evitar overfitting

# Añadir la capa de salida con softmax para la clasificación
model_transfer.add(Dense(train_generator.num_classes, activation='softmax'))

# Compilar el modelo
model_transfer.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Calcular pesos de clases para manejar clases desbalanceadas
from sklearn.utils import class_weight
import numpy as np

# Revisar si `train_generator.classes` contiene las clases correctas
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),  
    y=train_generator.classes  
)
class_weights = dict(enumerate(class_weights))  

history_transfer= model_transfer.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    class_weight=class_weights  
)

# Guardar el modelo entrenado
model_transfer.save('mejor_transfer_learning.h5')  

3.	Ajuste de Hiperparámetros y Fine-tuning:

    - Para el modelo CNN desde cero, ajustar el número de capas, neuronas por capa, tasa de aprendizaje y épocas de entrenamiento, esto hasta lograr los resultados esperados en términos de precisión (Accuracy).

    - Para el modelo de Transfer Learning, después de las primeras épocas de entrenamiento, descongelar algunas capas superiores del modelo preentrenado y realizar fine-tuning, ajustando el modelo a los datos deportivos, hasta lograr los resultados esperados en términos de precisión (Accuracy).

4.	Evaluación y Métricas:

    - Evaluar ambos modelos utilizando métricas como precisión, recall, F1-score y la pérdida en los datos de entrenamiento y validación.

    - Generar y presentar la matriz de confusión para analizar los errores de clasificación por cada deporte.

    - Graficar la evolución de la precisión y la pérdida durante el entrenamiento y la validación de ambos modelos.

    - Predecir de forma correcta al menos 6 de las 10 imágenes establecidas en la carpeta de single_test.

In [ ]:
# Evaluar el modelo CNN y obtener las predicciones
Y_pred_cnn = model_cnn.predict(validation_generator, steps=len(validation_generator), verbose=1)
y_pred_cnn_classes = np.argmax(Y_pred_cnn, axis=1)  

# Verifica las dimensiones de los datos
print("Shape del validation_generator:", validation_generator.samples)
print("Shape de las predicciones del modelo:", Y_pred_cnn.shape)

# Evaluar el modelo de Transfer Learning (si lo tienes)
Y_pred_transfer = model_transfer.predict(validation_generator, steps=len(validation_generator), verbose=1)
y_pred_transfer_classes = np.argmax(Y_pred_transfer, axis=1)  

# Obtener las etiquetas verdaderas
y_true = validation_generator.classes

# Asegúrate de que las dimensiones de las predicciones coincidan con las etiquetas verdaderas
print("Dimensiones de las predicciones CNN:", y_pred_cnn_classes.shape)
print("Dimensiones de las etiquetas verdaderas:", y_true.shape)

# Asegúrate de que las dimensiones coincidan antes de continuar con la evaluación
if y_pred_cnn_classes.shape != y_true.shape:
    print("Las dimensiones no coinciden. Revisa la configuración del modelo o los datos de entrada.")
else:
    # Reporte de clasificación para el modelo CNN
    print("Reporte de clasificación para el modelo CNN:")
    print(classification_report(y_true, y_pred_cnn_classes))

    # Reporte de clasificación para el modelo de Transfer Learning
    print("Reporte de clasificación para el modelo de Transfer Learning:")
    print(classification_report(y_true, y_pred_transfer_classes))

In [ ]:
# Lista de etiquetas de las clases deportivas
class_labels = ['ajedrez', 'baloncesto', 'boxeo', 'disparo', 'esgrima', 'formula1', 'futbol', 'hockey', 'natacion', 'tenis']

# Matriz de confusión para el modelo CNN
conf_matrix_cnn = confusion_matrix(y_true, y_pred_cnn_classes)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_cnn, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Matriz de Confusión CNN desde cero')
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.show()

# Matriz de confusión normalizada
conf_matrix_cnn = confusion_matrix(y_true, y_pred_cnn_classes, normalize='true')
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_cnn, annot=True, fmt='.2f', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Matriz de Confusión VGG16 ')
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.show()

In [ ]:

def plot_training_history(history, model_name):
    # Graficar la precisión
    plt.figure(figsize=(12, 6))

    # Subplot para la precisión
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Entrenamiento')
    plt.plot(history.history['val_accuracy'], label='Validación')
    plt.title(f'Evolución de la Precisión - {model_name}')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    # Subplot para la pérdida
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Entrenamiento')
    plt.plot(history.history['val_loss'], label='Validación')
    plt.title(f'Evolución de la Pérdida - {model_name}')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    # Mostrar las gráficas
    plt.tight_layout()
    plt.show()

# Llama a la función para graficar ambos modelos
plot_training_history(history_cnn, 'Modelo CNN desde cero')
plot_training_history(history_transfer, 'Modelo Transfer Learning')


In [ ]:
#Cargar y preprocesar la imagen para que coincida con el tamaño esperado por el modelo
test_image = load_img('C:/Users/mally/Documents/Proyectos/sem_IA2/CNN/single_test/single_test/tenis.jpg', target_size=(128, 128))
test_image = img_to_array(test_image)

# Normalizar la imagen como en el entrenamiento
test_image = test_image / 255.0

# Expandir las dimensiones para hacer la predicción (1, 128, 128, 3)
test_image = np.expand_dims(test_image, axis=0)

# Predecir la clase
result = model_transfer.predict(test_image)  # Aquí va el nombre del modelo que crearon

# Mostrar los valores predichos (probabilidades para cada clase)
print("Probabilidades predichas:", result)

# Obtener el índice de la clase con mayor probabilidad
predicted_class_index = np.argmax(result)

# Obtener el mapeo de clases
class_labels = train_generator.class_indices  # Cambiado a train_generator

# Invertir el diccionario para obtener las clases por índice
class_labels = dict((v, k) for k, v in class_labels.items())

# Obtener el nombre de la clase predicha
predicted_class_label = class_labels[predicted_class_index]

# Imprimir la clase predicha
print(f'La imagen pertenece a la clase: {predicted_class_label}')